<h1>Laboratorio N° 02 - Utilizando Operadores de Conjuntos de Datos

<h2>Ejercicio 1: Operador UNION

1.1. Conectar con la BD TSQL

In [2]:
USE TSQL;
GO

Commands completed successfully.

Total execution time: 00:00:00.005

1.2. Write a SELECT statement to return the productid and productname columns from the Production.Products table. Filter the results to include only products that have a categoryid value 4

In [3]:
SELECT productid, productname from Production.Products WHERE categoryid = 4

(10 rows affected)

Total execution time: 00:00:01.596

productid,productname
11,Product QMVUN
12,Product OSFNS
31,Product XWOXC
32,Product NUNAW
33,Product ASTMN
59,Product UKXRI
60,Product WHBYK
69,Product COAXA
71,Product MYMOI
72,Product GEEOO


1.3. Write a SELECT statement to return the productid and productname columns from the Production.Products table. Filter the results to include only products that have a total sales amount of more than $50,000. For the total sales amount, you will need to query the Sales.OrderDetails table and aggregate all order line values (qty * unitprice) for each product

In [4]:
SELECT 
    c.productid,
    c.productname,
    c.Sales_Amount
FROM
(
    SELECT 
	S.productid,P.productname, SUM(S.qty*S.unitprice) AS Sales_Amount
	FROM Sales.OrderDetails AS S
	INNER JOIN Production.Products AS P ON S.productid = P.productid
	GROUP BY S.productid,P.productname
) AS c 
WHERE c.Sales_Amount > 50000

(4 rows affected)

Total execution time: 00:00:00.013

productid,productname,Sales_Amount
29,Product VJXYN,87736.4000
38,Product QDOMO,149984.2000
59,Product UKXRI,76296.0000
60,Product WHBYK,50286.0000


1.3. Write a SELECT statement to retrieve the custid and contactname columns from the Sales.Customers table. Display the top 10 customers by sales amount for January 2008 and display the top 10 customers by sales amount for February 2008. (Hint: write two SELECT statements, each  joining Sales.Customers and Sales.OrderValues, and use the appropriate set operator.)

In [4]:
SELECT 
	TOP(10) 
	A.custid,A.contactname 
FROM
(	SELECT TOP(10)
		C.custid,C.contactname, (qty*val) as Sales_Amount 
	FROM Sales.Customers AS C INNER JOIN Sales.OrderValues AS S ON S.custid = C.custid
	WHERE MONTH(orderdate)=2 and YEAR(orderdate)=2008
	ORDER BY Sales_Amount DESC
) AS A
UNION ALL
SELECT 
	TOP(10) 
	A.custid,A.contactname 
FROM
(	SELECT TOP(10)
		C.custid,C.contactname, (qty*val) as Sales_Amount 
	FROM Sales.Customers AS C INNER JOIN Sales.OrderValues AS S ON S.custid = C.custid
	WHERE MONTH(orderdate)=1 and YEAR(orderdate)=2008
	ORDER BY Sales_Amount DESC
) AS A

(20 rows affected)

Total execution time: 00:00:00.036

custid,contactname
63,"Veronesi, Giorgio"
20,"Kane, John"
37,"Crăciun, Ovidiu V."
65,"Moore, Michael"
39,"Song, Lolan"
37,"Crăciun, Ovidiu V."
71,"Navarro, Tomás"
34,"Cohen, Shy"
72,"Welcker, Brian"
62,"Misiec, Anna"


<h2>Ejercicio 2: Operador EXCEPT y INTERSECT

2.1. Write a SELECT statement to retrieve the custid column from the Sales.Orders table. Filter the results to include only customers who bought more than 20 different products (based on the productid column from the Sales.OrderDetails table)

In [6]:
SELECT 
	P.custid 
FROM 
(
	SELECT 
		B.custid,COUNT(productid) AS TOTAL 
	FROM Sales.OrderDetails AS A
	INNER JOIN Sales.Orders AS B
	ON B.orderid = A.orderid
	GROUP BY A.orderid,B.custid
)
AS P  
WHERE TOTAL > 20;

(1 row affected)

Total execution time: 00:00:00.010

custid
65


2.2. Write a SELECT statement to retrieve the custid column from the Sales.Orders table. Filter the results to include only customers from the country USA and exclude all customers from the previous (task 1) result. (Hint: use the EXCEPT operator and the previous query.)

In [8]:
SELECT B.custid FROM Sales.Customers AS A
INNER JOIN Sales.Orders AS B ON B.custid = A.custid
WHERE A.country LIKE 'USA'
GROUP BY B.custid,A.country
EXCEPT 
SELECT 
	P.custid
FROM 
(
	SELECT 
		B.custid,COUNT(productid) AS TOTAL 
	FROM Sales.OrderDetails AS A
	INNER JOIN Sales.Orders AS B
	ON B.orderid = A.orderid
	GROUP BY A.orderid,B.custid
)
AS P  
WHERE TOTAL > 20;

(12 rows affected)

Total execution time: 00:00:00.034

custid
32
36
43
45
48
55
71
75
77
78


2.3. Write a SELECT statement to retrieve the custid column from the Sales.Orders table. Filter only customers who have a total sales value greater than $10,000. Calculate the sales value using the qty and unitprice columns from the Sales.OrderDetails table

In [9]:
SELECT P.custid
FROM
(
SELECT 
	T.custid, sum(qty*val) as Sales_Amount 
FROM Sales.OrderValues AS C INNER JOIN Sales.Orders AS T ON T.custid = C.custid
GROUP BY T.custid
) AS P
WHERE P.Sales_Amount > 10000


(87 rows affected)

Total execution time: 00:00:00.018

custid
23
46
69
29
75
9
15
89
3
52


<h2>Ejercicio 3: Utilizando el Operador APPLY

3.1. Write a SELECT statement to retrieve the productid and productname columns from the Production.Products table. In addition, for each product, retrieve the last two rows from the Sales.OrderDetails table based on orderid number.

3.2. Use the CROSS APPLY operator and a correlated subquery. Order the result by the column productid

3.3. Ejecutar la sentencia siguiente para crear la función fnGetTop3ProductsForCustomer.

In [1]:
DROP FUNCTION IF EXISTS dbo.fnGetTop3ProductsForCustomer; 
GO 
CREATE FUNCTION dbo.fnGetTop3ProductsForCustomer (@custid AS INT) 
RETURNS TABLE AS 
RETURN 
SELECT TOP(3) d.productid, p.productname, SUM(d.qty * d.unitprice) AS totalsalesamount 
FROM Sales.Orders AS o 
INNER JOIN Sales.OrderDetails AS d ON d.orderid = o.orderid 
INNER JOIN Production.Products AS p ON p.productid = d.productid 
WHERE custid = @custid 
GROUP BY d.productid, p.productname 
ORDER BY totalsalesamount DESC;

3.4. Write a SELECT statement to retrieve the custid and contactname columns from the Sales.Customers table. Use the CROSS APPLY operator with the dbo.fnGetTop3ProductsForCustomer function to retrieve productid, productname, and totalsalesamount columns for each customer